In [158]:
import requests
from bs4 import BeautifulSoup as BS 
import pandas as pd
import cloudscraper as cs
import re
from lxml import etree

In [5]:
URL = "https://en.wikipedia.org/wiki/List_of_Grey%27s_Anatomy_episodes#Season_1_(2005)"
page = requests.get(URL)
soup = BS(page.text)

In [49]:
soup.find('a', text = '"Weekly Program Rankings"').get('href')

'https://web.archive.org/web/20081221202459/http://abcmedianet.com/web/dnr/dispDNR.aspx?id=032905_05'

In [55]:
links = [x.get('href') for x in soup.find_all('a', text = '"Weekly Program Rankings"')]
len(links)

75

In [57]:
scraper=cs.create_scraper()
cloudsoup=BS(scraper.get('https://www.imdb.com/title/tt0413573/episodes/?season=1').content)

In [58]:
imdb=cloudsoup.findAll('a',attrs={'class':'ipc-title-link-wrapper'})
imdb

[<a class="ipc-title-link-wrapper" href="/title/tt0592902/?ref_=ttep_ep1" tabindex="0"><div class="ipc-title__text">S1.E1 ∙ A Hard Day's Night</div></a>,
 <a class="ipc-title-link-wrapper" href="/title/tt0592923/?ref_=ttep_ep2" tabindex="0"><div class="ipc-title__text">S1.E2 ∙ The First Cut Is the Deepest</div></a>,
 <a class="ipc-title-link-wrapper" href="/title/tt0592926/?ref_=ttep_ep3" tabindex="0"><div class="ipc-title__text">S1.E3 ∙ Winning a Battle, Losing the War</div></a>,
 <a class="ipc-title-link-wrapper" href="/title/tt0592915/?ref_=ttep_ep4" tabindex="0"><div class="ipc-title__text">S1.E4 ∙ No Man's Land</div></a>,
 <a class="ipc-title-link-wrapper" href="/title/tt0592919/?ref_=ttep_ep5" tabindex="0"><div class="ipc-title__text">S1.E5 ∙ Shake Your Groove Thing</div></a>,
 <a class="ipc-title-link-wrapper" href="/title/tt0592909/?ref_=ttep_ep6" tabindex="0"><div class="ipc-title__text">S1.E6 ∙ If Tomorrow Never Comes</div></a>,
 <a class="ipc-title-link-wrapper" href="/title

In [59]:
one=[x.get('href').strip() for x in imdb]
one

['/title/tt0592902/?ref_=ttep_ep1',
 '/title/tt0592923/?ref_=ttep_ep2',
 '/title/tt0592926/?ref_=ttep_ep3',
 '/title/tt0592915/?ref_=ttep_ep4',
 '/title/tt0592919/?ref_=ttep_ep5',
 '/title/tt0592909/?ref_=ttep_ep6',
 '/title/tt0592924/?ref_=ttep_ep7',
 '/title/tt0592918/?ref_=ttep_ep8',
 '/title/tt0592925/?ref_=ttep_ep9']

In [61]:
# my function needs to: change the season number at the end of the url then scrape all of the episode hrefs and then print them
episode_links = []

#wouldn't work without the {}
imdb_link='https://www.imdb.com/title/tt0413573/episodes/?season={}'

#started with the example from this: https://isabella-b.com/blog/scraping-episode-imdb-ratings-tutorial/
for sn in range(1,20):
    url=imdb_link.format(sn)
    #this is what i did to pull just season 1 but having .content was throwing an error
    response=scraper.get(url)
    #w/o this it just pulls 20 epis for each season
    test = BS(response.text, 'html.parser')
    #what i did for s1
    links=test.findAll('a',attrs={'class':'ipc-title-link-wrapper'})
    epi_links=[x.get('href').strip() for x in links]
    for link in epi_links:
        episode_links.append(link)

In [63]:
len(episode_links)

420

In [64]:
#add https://www.imdb.com to the front
episode_links[0]

'/title/tt0592902/?ref_=ttep_ep1'

In [80]:
scraper=cs.create_scraper()
cloudsoup=BS(scraper.get(f'https://www.imdb.com{episode_links[0]}').content)

In [82]:
#rating
cloudsoup.findAll('span',attrs={'class':'sc-bde20123-1 cMEQkK'})[0].text

'8.3'

In [83]:
#title
cloudsoup.findAll('span',attrs={'class':'hero__primary-text'})[0].text

"A Hard Day's Night"

In [97]:
#air date
cloudsoup.findAll('li',text = re.compile('Episode aired(.*)'))[0].text.replace('Episode aired ', '')

'Mar 27, 2005'

In [100]:
#runtime
cloudsoup.findAll('li',text = re.compile('(.*)m'))[0].text.replace('m', '')

'44'

In [101]:
#ep description
cloudsoup.findAll('span',attrs={'data-testid':'plot-xs_to_m'})[0].text

'The first shift for new surgical interns Meredith, Cristina, Izzie, George, and Alex proves eventful and backbreaking.'

In [102]:
#director
cloudsoup.findAll('a',attrs={'class':'ipc-metadata-list-item__list-content-item'})[0].text

'Peter Horton'

In [103]:
#writer
cloudsoup.findAll('a',attrs={'class':'ipc-metadata-list-item__list-content-item'})[1].text

'Shonda Rhimes'